### Replicamos los recursos locales en el entorno virtual

### Descargamos los datos de la competencia al entorno de trabajo

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 728MB/s]


### Descomprimimos el archivo

In [2]:
!unzip udea*.zip > /dev/null

In [3]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


## Importe de librerías necesarias

Se importan las librerías necesarias tales como:

- Pandas: Para la manipulación y análisis de datos con los DataFrames.
- Numpy: Para los cálculos matemáticos y el manejo eficiente de los arrays.
- Unidecode: Para normalizar el texto eliminado caracteres especiales o acentos.
- sklearn.preprocessing: Para convertir datos categóricos en un formato que nos sea de mayor utilidad.

In [4]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.6 MB/s eta 0:00:00


In [5]:
import unidecode as ud
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

### Cargamos el csv como un dataframe de Pandas
Guardamos el cvs en la variable test e imprimimos la cantidad de filas y columnas del DataFrame test.

In [6]:
test = pd.read_csv("test.csv")
print ("shape of loaded dataframe", test.shape)

shape of loaded dataframe (296786, 20)


Visualizamos las columnas del DataFrame test.

In [7]:
test.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'coef_1', 'coef_2',
       'coef_3', 'coef_4'],
      dtype='object')

Generamos una vista inicial de los datos.

In [8]:
test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


## One_hot encoding para la columna 'ESTU_PRGMACADEMICO'

Estrategia de preprocesado de 'ESTU_PRGM_ACADEMICO'
- Observamos cuantas carreras únicas hay y cuales de estas son.
- Las pasamos todas a minúsculas para que se agrupen algunas que eran las mismas pero escritas con diferentes combinaciones de mayúsculas y minúsculas.
- Dejamos en un array de numpy solo las carreras que tengan más de 6000 apariciones en el Dataframe original. Con el objetivo de agrupar al rededor de 20 carreras diferentes.
- A las demás carreras las agrupamos en un grupo 'otros' con el objetivo de reducir el ruido en los datos y la posible variabilidad que puedan presentar.
- Aplicamos OneHotEncoding a los nuevos datos en la columna.

In [9]:
carreras_unicas = test['ESTU_PRGM_ACADEMICO'].unique()

In [10]:
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: ud.unidecode(x.lower()))

In [11]:
test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,550236,20183,trabajo social,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,administracion comercial y de mercadeo,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,ingenieria mecatronica,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,contaduria publica,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,administracion de empresas,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [12]:
prgm_acads = ['administracion de empresas', 'derecho', 'contaduria publica',
       'psicologia', 'ingenieria industrial', 'ingenieria civil', 'medicina',
       'ingenieria de sistemas', 'licenciatura en pedagogia infantil',
       'ingenieria ambiental', 'trabajo social', 'enfermeria', 'arquitectura',
       'administracion en salud ocupacional', 'negocios internacionales',
       'economia', 'ingenieria mecanica', 'ingenieria electronica',
       'comunicacion social', 'administracion financiera']
len(prgm_acads)

20

In [13]:
array = [element for element in test['ESTU_PRGM_ACADEMICO'] if element not in prgm_acads ]
array

condition = test['ESTU_PRGM_ACADEMICO'].isin(array)
test.loc[condition, 'ESTU_PRGM_ACADEMICO'] = "otros"
test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4
0,550236,20183,trabajo social,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,otros,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,otros,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,contaduria publica,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,administracion de empresas,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [14]:
test['ESTU_PRGM_ACADEMICO'].value_counts()

,count
ESTU_PRGM_ACADEMICO,
otros,107041
administracion de empresas,33008
derecho,22984
contaduria publica,22103
psicologia,21256
ingenieria industrial,15228
ingenieria civil,9079
medicina,7915
ingenieria de sistemas,7655


In [15]:
encode = OneHotEncoder()

codificadorPrg = encode.fit_transform(test[['ESTU_PRGM_ACADEMICO']])

arreglo = pd.DataFrame(codificadorPrg.toarray(), columns = encode.categories_)
print(arreglo)
test.drop('ESTU_PRGM_ACADEMICO', axis = 1)

test = pd.concat([test, arreglo], axis = 'columns')
test['ESTU_PRGM_ACADEMICO']

       administracion de empresas administracion en salud ocupacional  \
0                             0.0                                 0.0   
1                             0.0                                 0.0   
2                             0.0                                 0.0   
3                             0.0                                 0.0   
4                             1.0                                 0.0   
...                           ...                                 ...   
296781                        1.0                                 0.0   
296782                        0.0                                 0.0   
296783                        0.0                                 0.0   
296784                        0.0                                 0.0   
296785                        0.0                                 0.0   

       administracion financiera arquitectura comunicacion social  \
0                            0.0          0.0         

,ESTU_PRGM_ACADEMICO
0,trabajo social
1,otros
2,otros
3,contaduria publica
4,administracion de empresas
...,...
296781,administracion de empresas
296782,derecho
296783,derecho
296784,otros


In [16]:
test = test.drop('ESTU_PRGM_ACADEMICO', axis = 1)

In [17]:
test.head()

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,550236,20183,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,98545,20203,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,499179,20212,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,782980,20195,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,785185,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
test.columns

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                     'ESTU_PAGOMATRICULAPROPIO',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administrac

## Categorización ordinal de las columnas categóricas ordinales

Decidimos convertir las variables categóricas que tienen un orden natural en un formato número para facilitar su procesamiento.

In [20]:
test.columns

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                     'ESTU_PAGOMATRICULAPROPIO',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administrac

## Quitarle los valores nulos a todas las columnas categóricas ordinales


### Conteo de valores nulos

Realizamos un conteo para saber cuantos valores nulos hay en algunas columnas específicas:

In [21]:
test['FAMI_EDUCACIONMADRE'].value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,60649
Primaria incompleta,42149
Técnica o tecnológica completa,38571
Educación profesional completa,36477
Secundaria (Bachillerato) incompleta,34566
Primaria completa,24399
Postgrado,20049
Técnica o tecnológica incompleta,11650
Educación profesional incompleta,9647


In [22]:
print(f"Cantidad nulos valor matrícula = {test['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()}")
print(f"Cantidad nulos horas semanales trabaja = {test['ESTU_HORASSEMANATRABAJA'].isna().sum()}")
print(f"Cantidad nulos estrato vivienda = {test['FAMI_ESTRATOVIVIENDA'].isna().sum()}")
print(f"Cantidad nulos educación padre = {test['FAMI_EDUCACIONPADRE'].isna().sum()}")
print(f"Cantidad nulos educación madre = {test['FAMI_EDUCACIONMADRE'].isna().sum()}")

Cantidad nulos valor matrícula = 2723
Cantidad nulos horas semanales trabaja = 13379
Cantidad nulos estrato vivienda = 13795
Cantidad nulos educación padre = 9993
Cantidad nulos educación madre = 10223


### Cambio de valores nulos por 'no info'

Realizamos este proceso con el fin de no perder información, indicar ausencia de datos y evitar sesgos conservando la distinción de que falta información.

In [23]:
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("no info")
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].fillna("no info")
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].fillna("no info")
test['FAMI_EDUCACIONPADRE'] = test['FAMI_EDUCACIONPADRE'].fillna("no info")
test['FAMI_EDUCACIONMADRE'] = test['FAMI_EDUCACIONMADRE'].fillna("no info")

## Codificación de valores categóricos ordinales.

Se define un conjunto de categorías para cada una de las columnas categóricas ordinales en el DataFrame con un orden lógico para transformar las columnas en valores numéricos, lo cuál es necesario para realizar nuestro modelo.

In [24]:
categories = [
    ['no info', 'No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón',
     'Entre 1 millón y menos de 2.5 millones', 'Entre 2.5 millones y menos de 4 millones',
     'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
     'Más de 7 millones'],  # ESTU_VALORMATRICULAUNIVERSIDAD
    ['no info', '0', 'Menos de 10 horas', 'Entre 11 y 20 horas', 'Entre 21 y 30 horas', 'Más de 30 horas'],  # ESTU_HORASSEMANATRABAJA
    ['no info', 'Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6'],  # FAMI_ESTRATOVIVIENDA
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONPADRE
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONMADRE
]



Utilizamos OrdinalEncoder de Scikitlearn para que a cada categoría dentro de la columna se le asigne un valor numperivo basado en el orden que se definió en el paso anterior.

In [25]:
codificador = OrdinalEncoder(categories=categories)
test[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE']] = codificador.fit_transform(
    test[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
           'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
           'FAMI_EDUCACIONMADRE']]
       )

In [26]:
test.head()

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,550236,20183,BOLIVAR,2.0,2.0,4.0,Si,9.0,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,98545,20203,ANTIOQUIA,5.0,4.0,3.0,Si,7.0,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,499179,20212,BOGOTÁ,4.0,1.0,4.0,Si,6.0,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,782980,20195,SUCRE,4.0,4.0,2.0,No,4.0,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,785185,20212,ATLANTICO,5.0,3.0,3.0,Si,7.0,Si,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
test.columns

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                     'ESTU_PAGOMATRICULAPROPIO',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administrac

Reemplazamos los valores nulos de la columna con el valor no info.

In [28]:
test['ESTU_PAGOMATRICULAPROPIO'] = test['ESTU_PAGOMATRICULAPROPIO'].fillna('no info')
test['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,163869
Si,130110
no info,2807


In [29]:
print(f"Cantidad nulos tiene internet = {test['ESTU_PAGOMATRICULAPROPIO'].isna().sum()}")
test['FAMI_TIENEINTERNET'].value_counts()

Cantidad nulos tiene internet = 0


,count
FAMI_TIENEINTERNET,
Si,254075
No,31172


## Conversión One-Hot de las columnas FAMI_TIENEINTERNET y ESTU_PAGOMATRICULAPROPIO

Preparamos los datos para hacer el uso del modelo con un one-hot.




In [30]:
one_hot_encoded = pd.get_dummies(test['ESTU_PAGOMATRICULAPROPIO'], prefix = 'ESTU_PAGOMATRICULAPROPIO')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,ESTU_PAGOMATRICULAPROPIO_no info
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
296781,0,1,0
296782,1,0,0
296783,1,0,0
296784,1,0,0


In [31]:
test = pd.concat([test, one_hot_encoded], axis = 1)
test = test.drop('ESTU_PAGOMATRICULAPROPIO', axis = 1)

In [32]:
test['ESTU_PAGOMATRICULAPROPIO_No'].head()

,ESTU_PAGOMATRICULAPROPIO_No
0,0
1,1
2,1
3,1
4,1


In [33]:
print(test.columns)

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('a

In [34]:
one_hot_encoded = pd.get_dummies(test['FAMI_TIENEINTERNET'], prefix = 'FAMI_TIENEINTERNET')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,0,1
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
296781,0,1
296782,0,1
296783,0,1
296784,0,1


Concatenamos las columnas One-hot al DataFrame original y eliminamos la columna original ya que no es necesaria, y así evitamos redundancia en el conjunto de datos

In [35]:
test = pd.concat([test, one_hot_encoded], axis = 1)
test = test.drop('FAMI_TIENEINTERNET', axis = 1)

Finalmente, realizamos una inspeccion y revision de las nuevas columnas generadas.

In [36]:
test['FAMI_TIENEINTERNET_Si'].head()

,FAMI_TIENEINTERNET_Si
0,1
1,1
2,1
3,0
4,1


### Eliminamos la columna de departamento para no tenerla en cuenta para este preprocesado

Decidimos agrupar los departamentos en categorías de 'ricos' y 'pobres', ya que creemos que esta clasificación tiene un impacto en el desempeño de los estudiantes. Las condiciones socioeconómicas de cada región influyen significativamente en el tipo de educación que reciben los estudiantes, lo que a su vez puede afectar sus resultados académicos. Dividimos los departamentos según información proporcionada por el DANE.

In [37]:
richest_deps = ['BOGOTÁ', 'RISARALDA', 'SANTANDER', 'ANTIOQUIA', 'CALDAS', 'CUNDINAMARCA', 'META', 'VALLE', 'QUINDIO', 'ATLANTICO', 'BOYACA']

In [38]:
test.columns

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                    

In [39]:
test['ESTU_PRGM_DEPARTAMENTO'] = test['ESTU_PRGM_DEPARTAMENTO'].apply(lambda x: 1 if x in richest_deps else 0)

In [40]:
test['ESTU_PRGM_DEPARTAMENTO'].values

array([0, 1, 1, ..., 1, 1, 1])

In [41]:
test.columns

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                    

### Eliminamos la columna periodo y ID ya que se considera, no tiene efecto sobre el rendimiento

In [42]:
test = test.drop('PERIODO', axis = 1)

In [43]:
test.head()

,ID,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,FAMI_TIENECOMPUTADOR,...,"(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)",ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,ESTU_PAGOMATRICULAPROPIO_no info,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,550236,0,2.0,2.0,4.0,9.0,Si,No,N,Si,...,0.0,0.0,0.0,0.0,1.0,0,1,0,0,1
1,98545,1,5.0,4.0,3.0,7.0,Si,No,N,Si,...,0.0,0.0,1.0,0.0,0.0,1,0,0,0,1
2,499179,1,4.0,1.0,4.0,6.0,Si,No,N,Si,...,0.0,0.0,1.0,0.0,0.0,1,0,0,0,1
3,782980,0,4.0,4.0,2.0,4.0,Si,No,N,No,...,0.0,0.0,0.0,0.0,0.0,1,0,0,1,0
4,785185,1,5.0,3.0,3.0,7.0,Si,No,N,Si,...,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1


In [44]:
test.shape

(296786, 42)

In [45]:
test.columns

Index([                                    'ID',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                              ('arquitectura',),
                    

In [46]:
test.to_csv('test2.csv', index = False)

In [47]:
!head test2.csv

ID,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4,"('administracion de empresas',)","('administracion en salud ocupacional',)","('administracion financiera',)","('arquitectura',)","('comunicacion social',)","('contaduria publica',)","('derecho',)","('economia',)","('enfermeria',)","('ingenieria ambiental',)","('ingenieria civil',)","('ingenieria de sistemas',)","('ingenieria electronica',)","('ingenieria industrial',)","('ingenieria mecanica',)","('licenciatura en pedagogia infantil',)","('medicina',)","('negocios internacionales',)","('otros',)","('psicologia',)","('trabajo social',)",ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,ESTU_PAGOMATRICULAPROPIO_no info,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
550236,0,2.0,2.0,4.0,9.0,Si,No,N,Si,Si,5.0,0.328,0.